In [25]:
import pandas as pd
import requests
import string
import urllib
# import datadotworld as dw
import os
from scrapy import Selector

In [26]:
# os.makedirs('/test')

In [27]:
# Data file URL
url = 'https://www.bls.gov/cew/downloadable-data-files.htm'

#Pull in page content
r = requests.get(url)

#Make Selector item to scrape
sel = Selector(text = r.text)

# Select annual average file list
sing_files = sel.xpath('//*[@id="cew_naics_data"]/tbody//td[7]//@href').extract()

In [28]:
# Define list of state codes from BLS link and filter to code and name
# This will be a reference to filter the single files
state_list = pd.read_csv('https://data.bls.gov/cew/doc/titles/area/area_titles.csv')
state_list = state_list[state_list['area_title'].str.contains('-- Statewide',regex=False)]
state_list['area_title'] = state_list['area_title'].str.split(' --',n=1,expand=True)[0]
state_list['area_fips'] = state_list['area_fips'].astype('int64')
state_list

,area_fips,area_title
4,1000,Alabama
76,2000,Alaska
129,4000,Arizona
149,5000,Arkansas
229,6000,California
292,8000,Colorado
361,9000,Connecticut
374,10000,Delaware
388,12000,Florida
461,13000,Georgia


In [29]:
# Define list of industries from BLS
industry_list = pd.read_csv('https://data.bls.gov/cew/doc/titles/industry/industry_titles.csv')

# Winnow down to just six-digit values
six_digit = industry_list[industry_list['industry_code'].str.len()==6]

# Identify only the newspaper-related code: 511110
news = (six_digit[six_digit['industry_title']
                  .str.contains('Newspaper publishers',regex=False)]
       )

all = (industry_list[industry_list['industry_title']
                     .str.contains('Total, all industries',regex=False)]
      )

news_all = pd.concat([news,all])

In [30]:
sing_files

[]

In [31]:
id=0
dfs=[]
for x in range(0,len(sing_files)-1):
    
    qcew = (pd.read_csv(sing_files[id])
            .merge(state_list, how='inner', on='area_fips')
            .merge(news_all,how='inner',on='industry_code')
           )
    dfs.append(qcew)
    id+=1
    
df = pd.concat(dfs,sort=False,ignore_index=True).drop_duplicates().reset_index(drop = True)

ValueError: No objects to concatenate

In [ ]:
df.to_csv('newspaper_jobs_and_totals_by_state.csv')